## Version 1

Traditional Agents feel a lot like following a recipe. You go step by step in the same order every time: take the input, apply the logic, call the tool, give the output. It works well when the task is predictable, but it struggles the moment anything unexpected happens.

ReAct Agents (Reason + Act) behave more like how we handle real situations. They think, take an action, look at what happened, and rethink their plan if needed. They constantly check whether they should call a tool or just move on, and they improve their decision as new information comes in. This makes them much more flexible and much better at multi-step or messy tasks. React follows reason->act->observe cycle.


The core idea is simple:
Traditional Agents follow a straight line.
ReAct Agents follow a loop that learns from each step.

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
class AgentState(TypedDict):
    ''' add_messages is used to store the messages properly since the property of State is to update and remove previous values.
        add_messages help store those changes.'''
    messages : Annotated[list, add_messages]


In [ ]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

llm = init_chat_model("google_genai:gemini-2.5-flash-lite")

agentnode will act the llm brain of the agent. Takes in the messages and produces responses.

In [ ]:
def agent_node(state:AgentState):
    '''
    Reads the messages -> Passes those into llm -> updates the state with the response from llm
    '''
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages":response}

In [ ]:
from langgraph.graph import StateGraph , START, END

builder = StateGraph(AgentState)

# builder.add_node("agent",agent_node)
# builder.add_edge(START,"agent")
# builder.add_edge("agent",END)
# graph = builder.compile()

In [ ]:
# out =graph.invoke({
#     "messages": ['hello','how are you']
# })

In [ ]:
# out

In [ ]:
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool

# # temporary: no real tools, empty list
# tools = []

# tool_node = ToolNode(tools)

# builder.add_node("tools", tool_node)

In [ ]:
# builder.compile()

Now we will use a conditional edge to check if the agent is calling tool or not. If tool is being called, we will use tool node to call the tool and update the messages in the state. If not, we will return the final response.

In [ ]:
def should_continue(state:AgentState):
    '''
    Check the last message -> if llm wants to call tools then return 'tools' else dont call
    '''
    last_message = state["messages"][-1]
    # langchain messages usually have .tool_calls when the llm wants tools
    if last_message.tool_calls:
        return 'tools'
    return END


In [ ]:
# builder.add_conditional_edges(
#     "agent",
#     should_continue,
#     ["tools",END]
# )

In [ ]:
# builder.add_edge("tools","agent")

In [ ]:
# builder.compile()

## Version 2

In [ ]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(max_results=2)

In [ ]:
out=search_tool.invoke("what is langchain")

In [ ]:
out

In [ ]:
from langchain_core.tools import tool


In [ ]:
#updating tools list

tools = [search_tool]
tool_node = ToolNode(tools)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

def agent_node(state: AgentState):
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    return {'messages':[response]}


In [ ]:
@tool
def save_note(content:str,tags:list[str]):
    '''
    Save the note to a file. This is done after getting response from the LLM.
    '''
    filename="research_notes.txt"
    with open(filename,'a') as f:
        f.write(f"\n--- NOTE ---\n")
        f.write(f"TAGS: {', '.join(tags)}\n")
        f.write(f"CONTENT: {content}\n")
        f.write(f"------------\n")
    return f"Note saved to {filename}"


In [ ]:
#update the tools node
tools = [search_tool,save_note]
llm_with_tools = llm.bind_tools(tools)
tool_node =ToolNode(tools) 

In [ ]:
builder.add_node("agent",agent_node)
builder.add_node("tools",tool_node)
builder.add_edge(START,"agent")
builder.add_conditional_edges(
    "agent",
    should_continue,
    ["tools",END]
)
builder.add_edge("tools","agent")
graph = builder.compile()

In [ ]:
graph

In [ ]:
initial_state = {
    "messages" :[("user","Research 'Langchain' and save a short summary with tags 'AI', 'Python'.")]
}

In [ ]:
for event in graph.stream(input=initial_state):
    for key,value in event.items():
        try:
            print(f"\nNode {key} finished.")
            print(value["messages"][-1].content)
        except Exception as e:
            print(f"{e}")